In [1]:
# IMPORTING ALL THE FUNDAMENTAL LIBRARIES WHICH ARE REQUIRED IN THIS PROJECT

import pandas as pd 
from pathlib import Path #used for taking path(pathlib,os,glob,etc.)
from concurrent.futures import ThreadPoolExecutor
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
#IMPORTING ALL THE LIBRARIES WHICH ARE USED IN MODEL MAKING(LAYERS,etc.)

import tensorflow as tf
from keras.layers import GlobalMaxPooling2D
from keras.layers import AveragePooling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import Concatenate
from keras.layers import Flatten
from keras.layers import Dense
from keras.applications import VGG16,VGG19,ResNet50,ResNet101
#from tf.keras.metrics import Accuracy, Precision, Recall

In [5]:
# Read the Dataset
complete_data_info=pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")

In [6]:
complete_data_info.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [7]:
complete_data_info.shape

(112120, 12)

In [9]:
# DataBasePath is stored
data_base_path = "/kaggle/input/data"

1.Now, we create two lists named as 'complete_data_imgs_paths' and 'imgs_file_names'.

2.We traverse through the database path and access the image index with suffix as .png. 

3.Append(to add) the whole image path in string format in complete_data_imgs_paths list.

4.And we search for a string(for last ultimate string) and append all the relative strings as string format in imgs_file_names(i.e. extension).

Here,we use two functions glob and parts, glob is used for searching of a specific pattern that is described inside the glob function in database, and parts is used to divide the image index path and access its last string.

In [10]:
complete_data_imgs_paths=list()
imgs_file_names=list()

for single_img_path in Path(data_base_path).glob("images_*/images/*.png"):
    
    complete_data_imgs_paths.append(str(single_img_path)) #pathlib.path.parts
    imgs_file_names.append(str(single_img_path.parts[-1]))

Here,we make a dataframe with two columns as 'Image Index' and 'Image Path' and give the 'imgs_file_names' to 'Image Index' and 'complete_data_imgs_paths' to 'Image Path'.
This dataframe gets stored in variable named as complete_data_path_info.

We did all these for future use.

In [13]:
complete_data_path_info=pd.DataFrame(data={"Image Index":imgs_file_names,
                                          "Image Path":complete_data_imgs_paths})

In [14]:
complete_data_path_info.head()

,Image Index,Image Path
0,00006199_010.png,/kaggle/input/data/images_003/images/00006199_...
1,00004833_016.png,/kaggle/input/data/images_003/images/00004833_...
2,00006260_000.png,/kaggle/input/data/images_003/images/00006260_...
3,00004911_010.png,/kaggle/input/data/images_003/images/00004911_...
4,00004186_007.png,/kaggle/input/data/images_003/images/00004186_...


Now, we merge the 'complete_data_path_info' which we created just above with 'complete_data_info' on the column index named as 'Image Index' with the merge() function and stored it in new variable named as complete_data_all_info.

In this new variable we have an extra column as Image Path which is not in complete_data_info.

In [15]:
#join the two tables using SQL command
complete_data_all_info=complete_data_info.merge(complete_data_path_info,on="Image Index")

In [16]:
complete_data_all_info.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000003_...


Now, we drop the unnecessary columns from the complete_data_all_info with drop() function.

In [17]:
complete_data_all_info.drop(complete_data_all_info.columns[2:6],axis=1,inplace=True)

In [18]:
complete_data_all_info.head()

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
0,00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000003_...


In [19]:
# fetching the testing data from complete data

testing_data_info=pd.read_csv("/kaggle/input/data/BBox_List_2017.csv")

In [20]:
testing_data_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN


In [21]:
testing_data_info.shape

(984, 9)

In [22]:
# Get the view of 'Image Index' column of testing_data_info
testing_data_info["Image Index"]

0      00013118_008.png
1      00014716_007.png
2      00029817_009.png
3      00014687_001.png
4      00017877_001.png
             ...       
979    00029464_015.png
980    00025769_001.png
981    00016837_002.png
982    00020124_003.png
983    00026920_000.png
Name: Image Index, Length: 984, dtype: object

Now,we get the length of unique rows of 'Image Index' column of testing_data_info, i.e.==880 and total==984

So, there are various rows or we can say that various diseases which are repeated.

In [30]:
len(testing_data_info["Image Index"].unique())

880

In [31]:
complete_data_all_info.head()

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
Image Index,,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_001/images/00000001_...
00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_001/images/00000002_...
00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000003_...


Here, we use the function set_index() which makes the column 'Image Index' as the index instead of labels(as 0,1,2...)

Now,we are free from the labels or serial nos.

We're doing this because testing_data_info also have labels so if we want merging then labeling of one should be removed.

In [32]:
complete_data_all_info=complete_data_all_info.set_index("Image Index")

KeyError: "None of ['Image Index'] are in the columns"

In [33]:
complete_data_all_info.head()

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
Image Index,,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_001/images/00000001_...
00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_001/images/00000002_...
00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000003_...


Here,we access the indexes or getting two rows by giving the Image Index using loc() function

In [34]:
complete_data_all_info.loc[["00000001_000.png","00000001_001.png"]]

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
Image Index,,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...


at below,we merge testing_data_info(which have indexes)with complete_data_all_info(which have not indexes) and stored in testing_data_path_info

In [35]:
testing_data_path_info=complete_data_all_info.loc[testing_data_info["Image Index"]]

In [36]:
testing_data_path_info.head()

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
Image Index,,,,,,,,
00013118_008.png,Atelectasis,PA,2992,2991,0.143,0.143,NaN,/kaggle/input/data/images_006/images/00013118_...
00014716_007.png,Atelectasis|Effusion|Mass,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_007/images/00014716_...
00029817_009.png,Atelectasis,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_012/images/00029817_...
00014687_001.png,Atelectasis|Cardiomegaly|Consolidation,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_007/images/00014687_...
00017877_001.png,Atelectasis,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_008/images/00017877_...


# Checking and accessing the repeating rows in testing_data_info(total=984),unique=880

In [37]:
testing_data_info[testing_data_info["Image Index"] == "00020124_003.png"]

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
594,00020124_003.png,Mass,361.244444,284.766823,106.951111,135.395556,NaN,NaN,NaN
982,00020124_003.png,Atelectasis,175.047118,580.456302,244.622222,103.537778,NaN,NaN,NaN


In [38]:
testing_data_info[testing_data_info["Image Index"] == "00014687_001.png"]

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
3,00014687_001.png,Atelectasis,726.237288,494.95142,141.016949,55.322034,NaN,NaN,NaN


In [39]:
testing_data_info[testing_data_info["Image Index"] == "00029464_015.png"]

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
322,00029464_015.png,Effusion,719.644444,611.309045,191.146667,51.200000,NaN,NaN,NaN
979,00029464_015.png,Atelectasis,198.940451,352.900747,615.537778,323.128889,NaN,NaN,NaN


In [40]:
complete_data_all_info.shape

(112120, 8)

In [41]:
testing_data_path_info.shape

(984, 8)

Now, we drop all the indexes in complete_data_all_info which have the indexes from testing_data_path_info 
and stored as training_data_all_info

In [42]:
training_data_all_info = complete_data_all_info.drop(index=testing_data_path_info.index)

In [43]:
training_data_all_info.shape

(111240, 8)

Now,we again reset the index for further use and give the indexes as 0,1,2...  of testing_data_path_info

In [44]:
testing_data_path_info.reset_index(inplace=True)

In [45]:
testing_data_path_info.head()

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
0,00013118_008.png,Atelectasis,PA,2992,2991,0.143,0.143,NaN,/kaggle/input/data/images_006/images/00013118_...
1,00014716_007.png,Atelectasis|Effusion|Mass,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_007/images/00014716_...
2,00029817_009.png,Atelectasis,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_012/images/00029817_...
3,00014687_001.png,Atelectasis|Cardiomegaly|Consolidation,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_007/images/00014687_...
4,00017877_001.png,Atelectasis,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_008/images/00017877_...


In [46]:
testing_data_info.shape

(984, 9)

We merged testing_data_info(collection of 984 rows) with testing_data_path_info(which we have just above reset) on the column named as 'Image Index' and stored as testing_data_all_info

In [47]:
testing_data_all_info = testing_data_info.merge(testing_data_path_info,
                                               on="Image Index")

In [48]:
testing_data_all_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN,Atelectasis,PA,2992,2991,0.143,0.143,NaN,/kaggle/input/data/images_006/images/00013118_...
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN,Atelectasis|Effusion|Mass,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_007/images/00014716_...
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN,Atelectasis,AP,3056,2544,0.139,0.139,NaN,/kaggle/input/data/images_012/images/00029817_...
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN,Atelectasis|Cardiomegaly|Consolidation,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_007/images/00014687_...
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN,Atelectasis,AP,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_008/images/00017877_...


AT THIS STAGE WE HAVE TRAINING_DATA_ALL_INFO & TESTING_DATA_ALL_INFO

In [49]:
testing_data_all_info.shape

(1216, 17)

# **concat dataframe**(for columns to be 984,not 1216(wrong figure occured))

In [50]:
testing_data_all_info=pd.DataFrame(testing_data_info,testing_data_path_info)

So we got the right figure as 984 because concatention does not add the rows but in merging this happens.

In [51]:
testing_data_all_info.shape

(984, 9)

In [52]:
testing_data_all_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
"(00013118_008.png, Atelectasis, PA, 2992, 2991, 0.143, 0.143, nan, /kaggle/input/data/images_006/images/00013118_008.png)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(00014716_007.png, Atelectasis|Effusion|Mass, AP, 3056, 2544, 0.139, 0.139, nan, /kaggle/input/data/images_007/images/00014716_007.png)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(00029817_009.png, Atelectasis, AP, 3056, 2544, 0.139, 0.139, nan, /kaggle/input/data/images_012/images/00029817_009.png)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(00014687_001.png, Atelectasis|Cardiomegaly|Consolidation, AP, 2500, 2048, 0.168, 0.168, nan, /kaggle/input/data/images_007/images/00014687_001.png)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(00017877_001.png, Atelectasis, AP, 2500, 2048, 0.168, 0.168, nan, /kaggle/input/data/images_008/images/00017877_001.png)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
testing_data_all_info.columns

Index(['Image Index', 'Finding Label', 'Bbox [x', 'y', 'w', 'h]', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')

Dropping the unnecessary columns in testing_data_all_info with the range.

In [55]:
testing_data_all_info.drop(labels=testing_data_all_info.columns[6:9],axis=1,inplace=True)

In [93]:
testing_data_all_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h]
"(00013118_008.png, Atelectasis, PA, 2992, 2991, 0.143, 0.143, nan, /kaggle/input/data/images_006/images/00013118_008.png)",NaN,NaN,NaN,NaN,NaN,NaN
"(00014716_007.png, Atelectasis|Effusion|Mass, AP, 3056, 2544, 0.139, 0.139, nan, /kaggle/input/data/images_007/images/00014716_007.png)",NaN,NaN,NaN,NaN,NaN,NaN
"(00029817_009.png, Atelectasis, AP, 3056, 2544, 0.139, 0.139, nan, /kaggle/input/data/images_012/images/00029817_009.png)",NaN,NaN,NaN,NaN,NaN,NaN
"(00014687_001.png, Atelectasis|Cardiomegaly|Consolidation, AP, 2500, 2048, 0.168, 0.168, nan, /kaggle/input/data/images_007/images/00014687_001.png)",NaN,NaN,NaN,NaN,NaN,NaN
"(00017877_001.png, Atelectasis, AP, 2500, 2048, 0.168, 0.168, nan, /kaggle/input/data/images_008/images/00017877_001.png)",NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
training_data_all_info.head()

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
0,00000001_000.png,[Cardiomegaly],PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,"[Cardiomegaly, Emphysema]",PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,"[Cardiomegaly, Effusion]",PA,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,[No Finding],PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,[Hernia],PA,2582,2991,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000003_...


So,this line of code splits the values in the "Finding Labels" column by the "|" character and updates the column with the resulting lists.

The lambda function lambda x: x.split("|") is used to split each element (presumably a string) in the "Finding Labels" column using the "|" (pipe) character as the separator. This results in a list of strings.

We're doing this in both the dataframes(training_data_all_info & testing_data_all_info)

In [98]:
training_data_all_info["Finding Labels"] = training_data_all_info["Finding Labels"].apply(lambda x: x if isinstance(x, list) else x.split("|"))
testing_data_all_info["Finding Label"] = testing_data_all_info["Finding Label"].apply(lambda x: x if isinstance(x, list) or pd.isna(x) else str(x).split("|"))


#training_data_all_info["Finding Labels"] = training_data_all_info["Finding Labels"].map(lambda x: x.split("|"))
#testing_data_all_info["Finding Label"] = testing_data_all_info["Finding Label"].map(lambda x: x.split("|"))
# map->lazy for loop->parallel for loop
#pool.map() ,not required sequence processing,all individual data is independent

Now,we create a list named as merged_list.

1.The loop iterates through each list in the "Finding Labels" column of the DataFrame training_data_all_info and concatenates these lists into a single list named merged_list. Essentially, it's creating a flat list that contains all the diseases from all the lists.

2.Here, the code converts merged_list into a set to get unique diseases and then back into a list. The line unique_diseases.remove("No Finding") removes the disease "No Finding" from the list of unique diseases. It seems like the code is excluding "No Finding" from the mapping.

3.Finally, last line uses the zip function to pair each unique disease with its corresponding index, starting from 0. The result is a dictionary disease2idx where keys are unique diseases, and values are numerical indices. This dictionary could be useful for encoding categorical labels into numerical values,


In [99]:
merged_list = list()

for single_list in training_data_all_info["Finding Labels"]:
    
    merged_list = merged_list + single_list

unique_diseases = set(merged_list)
unique_diseases = list(unique_diseases)
unique_diseases.remove("No Finding")
disease2idx = dict(zip(unique_diseases,range(0,len(unique_diseases))))

In [ ]:
 # diseases =complete_data_all_info["Finding Labels"].map(lambda x:x.split("|"))

In [ ]:
#type(diseases)

In [ ]:
#disease_list=list(diseases)

In [ ]:
#disease_set=set()
#for i in disease_list:
    
 #   disease_set.update(i)

In [ ]:
#disease_set

In [ ]:
#unique_diseases=list(disease_set)
#unique_diseases.remove("No Finding")

In [ ]:
#unique_diseases

In [ ]:
#disease2idx=dict(zip(unique_diseases,range(0,len(unique_diseases))))

In [ ]:
#disease2idx

Now, let's define a function named as map_diseases which takes input as disease_list.

1.It initializes an array all_zeros filled with zeros, where the length of the array is determined by the number of diseases in the disease2idx mapping.

2.It then iterates through each disease in the input list (disease_list).

3.For each disease, it checks if it is not equal to "No Finding". If true, it updates the corresponding index in the all_zeros array to 1.0.

4.The function returns the final binary-encoded array representing the presence (1.0) or absence (0.0) of diseases based on the input list.

In [111]:
# find the disease and set as 1(encoding)
def map_diseases(disease_list):
    
    all_zeros = np.zeros(len(disease2idx),)
    
    for single_disease in disease_list:
        
        if single_disease != "No Finding":
            all_zeros[disease2idx[single_disease]] = 1.0
        
    return all_zeros


Now,we

1.Use a ThreadPoolExecutor to parallelize the encoding process. The max_workers parameter is set to the number of CPU cores (os.cpu_count()), allowing concurrent execution of tasks.

2.Apply the map_diseases function to each element in the "Finding Labels" column of the training and testing data.

3.Convert the resulting list of binary-encoded arrays into a NumPy array.

*ThreadPoolExecutor is used to create a pool of worker threads. The number of workers is set to the number of CPU cores available (os.cpu_count()).

*Inside the with statement, a parallel execution of the map_diseases function is initiated for each element in the "Finding Labels" column of both the training and testing datasets.

*The pool.map method applies the map_diseases function to each element in the specified column concurrently, utilizing multiple threads.

*This parallelization can significantly speed up the encoding process, especially when dealing with large datasets, by taking advantage of multi-core processors.

In [118]:
with ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:

    multi_hot_encoded_Y_train = np.array(list(pool.map(map_diseases,training_data_all_info["Finding Labels"])))
    multi_hot_encoded_Y_test = np.array(list(pool.map(map_diseases,testing_data_all_info["Finding Label"])))

TypeError: 'float' object is not iterable

Now,we need to reset the index of training_data_all_info with change in real data

In [114]:
training_data_all_info.reset_index(inplace=True)

1.This part of the code is using the to_csv method of the DataFrame to export the data to a CSV file named "training_data.csv". The index=False argument specifies that the index column should not be included in the CSV file.

2.Same is also done for testing_data_all_info.

In [115]:
training_data_all_info.to_csv("training_data.csv",index=False)
testing_data_all_info.to_csv("testing_data.csv",index=False)

In [116]:
# Accessing rows from 0 to 2 in array multi_hot_encoded_Y_train 
multi_hot_encoded_Y_train[0:2]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [89]:
training_data_all_info["Finding Labels"].iloc[0:2]

0               [Cardiomegaly]
1    [Cardiomegaly, Emphysema]
Name: Finding Labels, dtype: object

In [91]:
testing_data_all_info["Finding Label"].iloc[0:2]

(00013118_008.png, Atelectasis, PA, 2992, 2991, 0.143, 0.143, nan, /kaggle/input/data/images_006/images/00013118_008.png)                 NaN
(00014716_007.png, Atelectasis|Effusion|Mass, AP, 3056, 2544, 0.139, 0.139, nan, /kaggle/input/data/images_007/images/00014716_007.png)   NaN
Name: Finding Label, dtype: float64

In [117]:
multi_hot_encoded_Y_test[0:2]

NameError: name 'multi_hot_encoded_Y_test' is not defined

SHUFFLING DATA:

1.b_revol = training_data_all_info.index: Creates a list of indices corresponding to the rows of training_data_all_info.

2.np.random.shuffle(b_revol): Shuffles the indices randomly.

3.training_data_all_info = training_data_all_info.iloc[b_revol]: Shuffles the rows of the DataFrame training_data_all_info according to the shuffled indices.

4.multi_hot_encoded_Y_train = multi_hot_encoded_Y_train[b_revol]: Presumably, shuffles the corresponding target labels.

GENERATOR LOOP:

1.for time_step in range(training_data_all_info.shape[0] // mb_size): Iterates over the number of mini-batches that can be created from the training data.

PROCESSING MINI-BATCH:

1.X_train_mb = list(): Initializes an empty list to store the images in the mini-batch.

2.The nested loop processes each image path in the current mini-batch, resizes the corresponding image to 1024x1024 pixels, and appends it to X_train_mb.

3.X_train_mb = np.array(X_train_mb): Converts the list of images into a numpy array.

4.Y_train_mb = multi_hot_encoded_Y_train[time_step * mb_size:(time_step + 1) * mb_size]: Extracts the corresponding target labels for the current mini-batch.

YIELDING MINI-BATCH:

1.yield X_train_mb, Y_train_mb: Yields the mini-batch of images (X_train_mb) and their corresponding target labels (Y_train_mb) to be used during model training.

In [132]:
def custom_training_data_generator(training_data_all_info,mb_size):
    b_revol = training_data_all_info.index.values
    np.random.shuffle(b_revol)
    training_data_all_info = training_data_all_info.reindex(b_revol)
    multi_hot_encoded_Y_train = multi_hot_encoded_Y_train[b_revol]

   # b_revol=training_data_all_info.index
   #np.random.shuffle(b_revol)
   #training_data_all_info=training_data_all_info.iloc[b_revol]
   #multi_hot_encoded_Y_train=multi_hot_encoded_Y_train[b_revol]
    
    
    for time_step in range(training_data_all_info.shape[0]//mb_size):
        
         X_train_mb=list()
    
        
         for single_img_path in trainig_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size]["Image Path"]:
            
                resized_single_img=cv2.resize(Image.open(single_img_path).convert("RGB"),(1024,1024))
                X_train_mb.append(resized_single_img)
                    
                X_train_mb=np.array(X_train_mb)
                Y_train_mb=multi_hot_encoded_Y_train[time_step*mb_size:(time_step+1)*mb_size]
                    
                yield X_train_mb,Y_train_mb
    
    

In [134]:
for X_train_mb, Y_train_mb in custom_training_data_generator(training_data_all_info,40):
    
    print(X_train_mb.shape,Y_train_mb.shape)


UnboundLocalError: local variable 'multi_hot_encoded_Y_train' referenced before assignment

# 4 pretrained cnns

NOW WE USE PRE-TRAINED CNNS FOR TRANSFER LEARNING:-

THEY ALL ARE TRAINED ON SPECIFIC CV DATASETS.

In [135]:
from keras.applications import VGG16,VGG19,ResNet50,ResNet101

In [136]:
pretrained_vgg16_conv_base=VGG16(include_top=False,input_shape=(1024,1024,3),classes=14)
pretrained_vgg19_conv_base=VGG19(include_top=False,input_shape=(1024,1024,3),classes=14)
pretrained_resnet50_conv_base=ResNet50(include_top=False,input_shape=(1024,1024,3),classes=14)
pretrained_resnet101_conv_base=ResNet101(include_top=False,input_shape=(1024,1024,3),classes=14)


171446536/171446536 [==============================] - 1s 0us/step


# 1.Data Engineering-by pyspark and databricks
# 2.Data Science-hadoop(hdfs)
# 3.Model Deployment(MLops)

Now,we check or view ths summary of a pretrained network.

In [137]:
pretrained_vgg16_conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1024, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 1024, 1024, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 1024, 1024, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 512, 512, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 512, 512, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 512, 512, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 256, 256, 128)     0     

we should change the trainable parameters to non trainable(because we may change trainable by gradient descent)

In [138]:
pretrained_vgg16_conv_base.trainable=False
pretrained_vgg19_conv_base.trainable=False
pretrained_resnet50_conv_base.trainable=False
pretrained_resnet101_conv_base.trainable=False

In [139]:
pretrained_vgg16_conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1024, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 1024, 1024, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 1024, 1024, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 512, 512, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 512, 512, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 512, 512, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 256, 256, 128)     0     

In [140]:
pretrained_vgg19_conv_base.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1024, 1024, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 1024, 1024, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 1024, 1024, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 512, 512, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 512, 512, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 512, 512, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 256, 256, 128)     0     

In [142]:
pretrained_resnet50_conv_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1024, 1024,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 1030, 1030,   0           ['input_3[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, 512, 512, 64  9472        ['conv1_pad[0][0]']              
                                )                                                          

In [143]:
pretrained_resnet101_conv_base.summary()

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 1024, 1024,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 1030, 1030,   0           ['input_4[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, 512, 512, 64  9472        ['conv1_pad[0][0]']              
                                )                                                         

HW:-make a model whose output is (40,32,32,512+512+2048+2048, tensorflow tensor) by functional api(model operator)

**example of dense layer and globallse**:

KEY ELEMENTS:
class GlobalLSEPooling2D(tf.keras.layers.layer):
    def_init_(self,layer_parametrs):
        super(Globallsepoo.,self)
        self.layer_inputs=layer_parametrs.

This code defines a custom testing data generator function named custom_testing_data_generator. The generator is designed to yield batches of testing data during model evaluation, likely for a neural network trained on images with associated bounding boxes and labels.

1.Loop Over Mini-Batches:

The outer loop (for time_step in range(testing_data_all_info.shape[0] // mb_size)) iterates over the mini-batches in the testing dataset.

2.Processing Mini-Batch:

The inner loop processes each image path in the current mini-batch, resizing the corresponding image to 1024x1024 pixels.

X_test_mb collects the resized images for the mini-batch.

3.Extracting Labels:

Y_test_mb is assigned the corresponding target labels for the current mini-batch.

4.Bounding Box and Centroid Calculation:

bbox_mb extracts bounding box information for the mini-batch.

centroid_mb calculates the centroid of each bounding box.

5.Image Size and Rescaled Centroid:

img_size_mb extracts image size information for the mini-batch.

ordered_img_size_mb reorders dimensions of the image size
.
rescaled_centroid_mb calculates rescaled centroids based on the image size.

6.Yielding Mini-Batch:

The generator yields the mini-batch containing resized images (X_test_mb), target labels (Y_test_mb), and rescaled centroids (rescaled_centroid_mb) for model evaluation.

In [145]:
def custom_testing_data_generator(testing_data_all_info, multi_hot_encoded_Y_test, mb_size):
    
    for time_step in range(testing_data_all_info.shape[0]//mb_size):
        
        X_test_mb = list()
    
        for single_img_path in testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size]["Image Path"]:
            
            resized_single_img =  cv2.resize(np.array(Image.open(single_img_path).convert("RGB")),(1024,1024))
            X_test_mb.append(resized_single_img)
            
        X_test_mb = np.array(X_test_mb)
        Y_test_mb = multi_hot_encoded_Y_test[time_step*mb_size:(time_step+1)*mb_size]
        
        bbox_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,2:6])
        centroid_mb = bbox_mb[:,0:2] + 0.5*np.concatenate((bbox_mb[:,3],bbox_mb[:,2]),axis=1)
        
        img_size_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,8:10])
        ordered_img_size_mb = np.concatenate((img_size_mb[:,1],img_size_mb[:,0]),axis=1)
        rescaled_centroid_mb = centroid_mb * (32/ordered_centroid_mb)
        
        yield X_test_mb, Y_test_mb, rescaled_centroid_mb

This is the Keras layer named GlobalLSEPooling2D.

This layer is designed to perform a modified form of Global Log-Sum-Exp (LSE) pooling operation on a 2D input tensor. LSE pooling is commonly used in neural network architectures as an alternative to other pooling techniques like max pooling or average pooling.

1.Initialization (__init__ method):

The layer is initialized with a hyperparameter r_hyper_param. This hyperparameter controls the sensitivity of the pooling operation.

2.call Method (Forward Pass):

The call method is where the forward pass of the layer is defined.

3.Global Max Pooling per Channel:

x_star_per_channel calculates the global max pooling operation along each channel of the input tensor (concatenated_input). The result is a tensor where each channel's maximum value is retained.

4.Exponential Shift and Scaling:

shifted_pix_values applies an exponential shift to each pixel value in concatenated_input based on the difference between the pixel value and the corresponding maximum value (x_star_per_channel). This operation is controlled by the hyperparameter r.

5.Global Average Pooling of Shifted Values:

avged_output performs global average pooling on the shifted pixel values. This step essentially calculates the average of the exponential-shifted values.

6.Logarithmic Scaling:

logged_output applies a logarithmic scaling to the averaged and shifted values. The scaling factor is (1 / self.r).

7.Final Layer Output:

layer_output is the sum of the original global max-pooled values (x_star_per_channel) and the logarithmically scaled values (logged_output).

8.Return Statement:

The method returns layer_output, which represents the output of the layer after the Global LSE pooling operation.

During the training and evaluation process, the weights of this layer will be adjusted based on the optimization objective of the overall model.

In [3]:
class GlobalLSEPooling2D(tf.keras.layers.Layer):
    
    def __init__(self,r_hyper_param):
        super(GlobalLSEPooling2D, self).__init__()
        self.r=r_hyper_param
        
        def call(self,concatenated_input):
            x_star_per_channel=GlobalMaxPooling2D(keepdims=True)(concatenated_input)
            shifted_pix_values=tf.math.exp(self.r*(concatenated_input - x_star_per_channel))
            avged_output=GlobalAveragePooling2D(keepdims=True)(shifted_pix_values)
            logged_output=(1/self.r)*tf.math.log(avged_output)
            layer_output=x_star_per_channel+logged_output
            
            return layer_output

This line of code generates a NumPy array random_output filled with random numbers sampled from a normal distribution.

np.random.normal: This is a NumPy function that generates random numbers from a normal distribution (also known as a Gaussian distribution) with a mean of 0 and a standard deviation of 1 by default. It can be customized with additional parameters to control the mean and standard deviation.

In [4]:
random_output=np.random.normal(size=(40,32,32,5120))

Now,we call GlobalLSEPooling2D with r=0.9 ans gives input as random_output ans stored as our_lse_layer_output

In [5]:
our_lse_layer_output=GlobalLSEPooling2D(r_hyper_param=0.9)(random_output)

In [6]:
# SHAPE OF THE LSE LAYER OUTPUT
our_lse_layer_output.shape

TensorShape([40, 32, 32, 5120])

Now, we define our chest_x_ray_cnn function which takes input shape as (1024,1024,3).

After that we take base of pretrained CNNs(not top) and make trainable parameters as false.

We use concatenate layer to concatenate all pretrained CNNs.

We also use GlobalLSEPooling2D and Flatten layer.

We use a layer named as Dense with sigmoid activation function
and put the return statement.


In [9]:
def chest_x_ray_cnn():
    
    input_to_cnn = Input(shape=(1024,1024,3))
    
    pretrained_vgg16_conv_base = VGG16(include_top=False,
                                   input_shape=(1024,1024,3))
    pretrained_vgg19_conv_base = VGG19(include_top=False,
                                  input_shape=(1024,1024,3))
    pretrained_resnet50_conv_base = ResNet50(include_top=False,
                                        input_shape=(1024,1024,3))
    pretrained_resnet101_conv_base = ResNet101(include_top=False,
                                          input_shape=(1024,1024,3))

    pretrained_vgg16_conv_base.trainable = False
    pretrained_vgg19_conv_base.trainable = False
    pretrained_resnet50_conv_base.trainable = False
    pretrained_resnet101_conv_base.trainable = False
    
    vgg16_out = pretrained_vgg16_conv_base(input_to_cnn)
    vgg19_out = pretrained_vgg19_conv_base(input_to_cnn)
    resnet50_out = pretrained_resnet50_conv_base(input_to_cnn)
    resnet101_out = pretrained_resnet101_conv_base(input_to_cnn)
    
    concatenated_output = Concatenate()([vgg16_out,vgg19_out,
                                      resnet50_out,resnet101_out])
    
    pooled_output = GlobalLSEPooling2D(r_hyper_param=0.9)(concatenated_output)
    flattened_output = Flatten()(pooled_output)
    
    cnn_out = Dense(units=multi_hot_encoded_Y_train.shape[1],
                    activation="sigmoid")(flattened_output)
    
    return Model(inputs=input_to_cnn,outputs=cnn_out)
    

Now stored the function chest_x_ray_cnn to our_custom_cnn.

In [10]:
our_custom_cnn=chest_x_ray_cnn()

171446536/171446536 [==============================] - 1s 0us/step


NameError: name 'multi_hot_encoded_Y_train' is not defined

In [11]:
# Viewing the summary
our_custom_cnn.summary()

NameError: name 'our_custom_cnn' is not defined

loss function always be a scalar value

This code defines a custom loss function named weighted_bcel (weighted binary cross-entropy loss) using TensorFlow operations.

1.Inputs:

Y_train_mb: Ground truth labels (binary) for the mini-batch.
Y_pred_mb: Predicted probabilities for each class in the mini-batch.

2.Cardinality Calculation:

cardinality_p: Number of positive instances in the mini-batch.
cardinality_n: Number of negative instances in the mini-batch (total instances minus positive instances).

3.Beta Calculation:

beta_p: Weighting factor for positive class. It is calculated as the sum of positive and negative cardinalities divided by the positive cardinality.
beta_n: Weighting factor for negative class. It is calculated as the sum of positive and negative cardinalities divided by the negative cardinality.

4.Loss Calculation:

The loss is computed using the binary cross-entropy formula, with class-dependent weights (beta_p and beta_n).
tf.math.log(Y_pred_mb): Logarithm of the predicted probabilities.
tf.math.log(1-Y_pred_mb): Logarithm of the complement of the predicted probabilities.
beta_p*Y_train_mb*tf.math.log(Y_pred_mb): Weighted contribution of positive class.
beta_n*(1-Y_train_mb)*tf.math.log(1-Y_pred_mb): Weighted contribution of negative class.
tf.reduce_mean(tf.reduce_mean(..., axis=0)): Compute the mean loss across all classes and then take the mean across the mini-batch.

5.Return:

The negative of the computed loss is returned. The negative sign is used because most optimization algorithms minimize the loss, but in this case, the objective is to maximize the weighted binary cross-entropy.

In [ ]:
def weighted_bcel(Y_train_mb,Y_pred_mb):
    
    cardinality_p=np.count_nonzero(Y_train_mb)
    cardinality_n=(Y_train_mb.shape[0]*Y_train_mb.shape[1]-cardinality_p)
    
    beta_p=(cardinality_p+cardinality_n)/cardinality_p
    beta_n=(cardinality_p+cardinality_n)/cardinality_n 
    
   # Y_train_mb_not=1-Y_train_mb
   # Y_pred_mb_not=1-Y_pred_mb
    
    return -(tf.reduce_mean(tf.reduce_mean(beta_p*Y_train_mb*tf.math.log(Y_pred_mb)
                          + beta_n*(1-Y_train_mb)*tf.math.log(1-Y_pred_mb),axis=0)))


    
    


This code defines a function named compute_performance_metrics that computes various performance metrics for a binary classification problem.

1.Inputs:

Y_pred: Predicted binary labels.
Y_true: True binary labels.
thresh: Threshold for converting predicted probabilities into binary predictions. If the predicted probability is greater than thresh, it is considered as a positive prediction; otherwise, it is considered as a negative prediction.

2.Convert Predictions to Binary:

Y_pred = Y_pred > thresh: Convert predicted probabilities to binary predictions using the specified threshold.

3.Confusion Matrix Calculation:

multilabel_confusion_matrix: This function computes the confusion matrix for each class in a multilabel classification problem. In this case, it's applied to calculate the confusion matrix for binary classification.

4.Sum Confusion Matrices:

summed_confusion_matrix = np.sum(confusion_matrix, axis=0): Sum the confusion matrices along the first axis (axis=0), resulting in a combined confusion matrix.

5.Extract True Positives (tp), True Negatives (tn), False Positives (fp), and False Negatives (fn):

tp = summed_confusion_matrix[0, 0]: True positives.
tn = summed_confusion_matrix[1, 1]: True negatives.
fp = summed_confusion_matrix[0, 1]: False positives.
fn = summed_confusion_matrix[1, 0]: False negatives.

6.Compute Performance Metrics:

accuracy = (tp + tn) / (tp + tn + fp + fn): Accuracy is the ratio of correctly predicted instances to the total instances.
precision = tp / (tp + fp): Precision is the ratio of correctly predicted positive observations to the total predicted positives.
recall = tp / (tp + fn): Recall (Sensitivity) is the ratio of correctly predicted positive observations to the all observations in actual class.

7.Return Metrics:

The function returns the computed accuracy, precision, and recall.

In [ ]:
def compute_performance_metrics(Y_pred,Y_true,thresh):
    
    Y_pred = Y_pred > thresh
    confusion_matrix = multilabel_confusion_matrix(y_true=Y_true,y_pred=Y_pred)
    summed_confusion_matrix = np.sum(confusion_matrix,axis=0)
    
    tp = summed_confusion_matrix[0,0]
    tn = summed_confusion_matrix[1,1]
    fp = summed_confusion_matrix[0,1]
    fn = summed_confusion_matrix[1,0]
    
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    
    return accuracy,precision,recall    

This code defines a function named predicted_center_pixel_cords that takes a list of heatmaps as input and computes the predicted center pixel coordinates for each heatmap channel.

1.Inputs:

heatmaps: A list of heatmaps. Each heatmap is assumed to have multiple channels.

2.Initialize Empty List:

argmax = list(): Initialize an empty list to store the predicted center pixel coordinates for each heatmap.

3.Loop Over Heatmaps:

for single_heatmap in heatmaps:: Iterate through each heatmap in the input list.

4.Split Channels:

heatmap_channels = cv2.split(single_heatmap): Split each heatmap into its individual channels.

5.Loop Over Channels:

for single_heatmap_channel in heatmap_channels:: Iterate through each channel of the heatmap.

6.Find Argmax Location:

single_channel_max_loc = list(np.unravel_index(np.argmax(single_heatmap_channel), single_heatmap_channel.shape)): Find the location (row, column) of the maximum value in the current channel using np.argmax and np.unravel_index.

7.Append to List:

argmax_per_heatmap.append(single_channel_max_loc): Append the calculated maximum location for each channel to the argmax_per_heatmap list.

8.Append to Outer List:

argmax.append(np.array(argmax_per_heatmap) / np.array([[7,7]]) * 224): Append the list of maximum locations for each channel, normalized by dividing by [7, 7] and multiplied by 224, to the argmax list. This step is likely intended to convert the normalized coordinates back to the original image size (e.g., from a 7x7 grid to a 224x224 image).

9.Return Result:

return np.array(argmax): Return the final result as a NumPy array containing the predicted center pixel coordinates for each heatmap.

In [ ]:
def predicted_center_pixel_cords(heatmaps):
    
    argmax = list()
    for single_heatmap in heatmaps:
        
        heatmap_channels = cv2.split(single_heatmap)
        argmax_per_heatmap = list()
    
        for single_heatmap_channel in heatmap_channels:    
            single_channel_max_loc = list(np.unravel_index(np.argmax(single_heatmap_channel),single_heatmap_channel.shape))
            argmax_per_heatmap.append(single_channel_max_loc)
        
        argmax.append(np.array(argmax_per_heatmap)/np.array([[7,7]])*224)
        
    return np.array(argmax)

This code creates a new Keras model called conv_base by extracting a portion of an existing model (our_custom_cnn).

In summary, the code creates a new model (conv_base) that takes the same input as our_custom_cnn but outputs the activations of the layer at index 5 in our_custom_cnn. This can be useful, for example, in transfer learning scenarios where you want to use the pre-trained features of our_custom_cnn up to a certain layer in a different model.

In [ ]:
conv_base = Model(inputs=[our_custom_cnn.input],outputs=[our_custom_cnn.layers[5].output])

This code represents a training loop for a neural network using TensorFlow and Keras.

1.Optimizer Initialization:

optimizer = tf.keras.optimizers.RMSprop(): Initializes an RMSprop optimizer. This optimizer will be used to update the model weights during training.

2.Training Parameters:

num_epochs = 25: Specifies the number of training epochs.
time_steps = 0: Initializes a counter for time steps.
mb_size = 2: Defines the mini-batch size.

3.Training Loop:

The loop iterates over the specified number of epochs (num_epochs).
For each epoch, it iterates over mini-batches generated by the custom_training_data_generator.

4.Gradient Tape and Backpropagation:

with tf.GradientTape() as tape:: Sets up a context for automatic differentiation.
Y_pred_mb = our_custom_cnn(X_train_mb): Forward pass to obtain predictions from the model (our_custom_cnn) given input X_train_mb.
w_bcel_value = weighted_bcel(Y_train_mb, Y_pred_mb): Computes the weighted binary cross-entropy loss.
gradients = tape.gradient(w_bcel_value, our_custom_cnn.trainable_weights): Computes the gradients of the loss with respect to the trainable weights.
optimizer.apply_gradients(zip(gradients, our_custom_cnn.trainable_weights)): Updates the model weights using the optimizer.

5.Performance Metrics Calculation:

train_acc, train_pre, train_rec = compute_performance_metrics(Y_pred_mb, Y_train_mb, 0.5): Computes accuracy, precision, and recall for the current mini-batch.

6.Print Training Progress:

Prints training information, including the epoch, time step, weighted binary cross-entropy loss, training accuracy, precision, and recall.

7.Feature Extraction and Heatmap Calculation:

conv_base_out = conv_base(X_train_mb).numpy(): Extracts features from the convolutional base for the input mini-batch.
cls_head_params = our_custom_cnn.layers[-1].weights[0].numpy(): Retrieves the weights of the last layer of the model (our_custom_cnn).
heatmaps = np.matmul(conv_base_out, cls_head_params): Calculates heatmaps by multiplying the feature vectors with the last layer's weights.

8.Predicted Center Pixel Locations:

pred_center_pix_loc = predicted_center_pixel_cords(heatmaps): Computes predicted center pixel locations based on the heatmaps.

9.Print Predicted Locations:

Prints the predicted center pixel locations for each disease in the mini-batch.

In [ ]:
optimizer=tf.keras.optimizers.RMSprop()

num_epochs=25
time_steps=0
mb_size=2

for epoch in range(num_epochs):
    for X_train_mb,Y_train_mb in custom_training_data_generator(training_data_all_info,multi_hot_encoded_Y_train,mb_size):
        
        with tf.GradientTape() as tape:
            
            Y_pred_mb=our_custom_cnn(X_train_mb)
            w_bcel_value = weighted_bcel(Y_train_mb,Y_pred_mb)
            
            gradients=tape.gradient(w_bcel_value,our_custom_cnn.trainable_weights)
            optimizer.apply.gradient(zip(gradients,our_custom_cnn.trainable_weights))
            
            train_acc,train_pre,train_rec = compute_performance_metrics(Y_pred_mb,Y_train_mb,0.5)
        
        time_steps += 1
            
        print("\n\nEpoch # {}, Time Step # {}".format(epoch,time_steps))
        print("WBCEL Value = {}, Training Accuracy = {}, Training Precision = {}, Training Recall = {}".format(w_bcel_value,
                                                                                                              train_acc,
                                                                                                              train_pre,
                                                                                                              train_rec))
        conv_base_out = conv_base(X_train_mb).numpy()
        cls_head_params = our_custom_cnn.layers[-1].weights[0].numpy()
        
        heatmaps = np.matmul(conv_base_out,cls_head_params)
        pred_center_pix_loc = predicted_center_pixel_cords(heatmaps)
        
        print("Epoch # {}, Time Step # {}, Predicted Location of Each Disease for Chest X-rays in Mini Batch:".format(epoch,
                                                                                                                     time_steps))
        print(pred_center_pix_loc)

# **Going to perform K-fold cross validation to perform Hyperparameter Optimization of Threshold Probability.**

This code performs testing and evaluation of a model on multiple folds and threshold probabilities.

1.Print Statements:

print("\n\n\n\n"): Prints four new lines, creating some visual separation in the output.


2.Threshold Probability Range and Fold Size:

thresh_probability = np.arange(start=0.3, stop=0.8, step=0.1): Generates an array of threshold probabilities ranging from 0.3 to 0.8 in steps of 0.1.
fold_size = 123: Defines the size of each fold in the cross-validation.

3.Number of Folds (K):

K = testing_data_all_info.shape[0] // fold_size: Calculates the number of folds based on the size of the testing data and the specified fold size.

4.Cross-Validation Loop:

The outer loop (for fold_idx in range(K):) iterates over each fold in the cross-validation.

5.Threshold Probability Loop:

The inner loop (for p_thresh in thresh_probability:) iterates over each threshold probability.

6.Create Cross-Validation Data (cv_data):

cv_data = pd.concat([...], axis=0): Concatenates the testing data excluding the current fold to create the cross-validation data.

7.Generate Cross-Validation and Testing Data:

X_cv, Y_cv, rescaled_centroid_cv = custom_testing_data_generator(...): Generates cross-validation data and labels.
X_test, Y_test, rescaled_centroid_test = custom_testing_data_generator(...): Generates testing data and labels.

8.Model Predictions and Performance Metrics for Cross-Validation (cv):

Y_pred_cv = our_custom_cnn(X_cv): Computes model predictions on the cross-validation data.
cv_acc, cv_pre, cv_rec = compute_performance_metrics(Y_pred_cv, Y_cv, p_thresh): Computes performance metrics (accuracy, precision, and recall) on the cross-validation data using the specified threshold probability.

9.Feature Extraction and Heatmap Calculation for Cross-Validation (cv):

cv_conv_base_out = conv_base(X_cv).numpy(): Extracts features from the convolutional base for the cross-validation data.
cv_heatmaps = np.matmul(cv_conv_base_out, cls_head_weights): Calculates heatmaps by multiplying the feature vectors with the last layer's weights.

10.Model Predictions and Performance Metrics for Testing (test):

Y_pred_test = our_custom_cnn(X_test): Computes model predictions on the testing data.
test_acc, test_pre, test_rec = compute_performance_metrics(Y_pred_test, Y_test, p_thresh): Computes performance metrics (accuracy, precision, and recall) on the testing data using the specified threshold probability.

11.Feature Extraction and Heatmap Calculation for Testing (test):

test_conv_base_out = conv_base(X_test).numpy(): Extracts features from the convolutional base for the testing data.
test_heatmaps = np.matmul(test_conv_base_out, cls_head_weights): Calculates heatmaps by multiplying the feature vectors with the last layer's weights.

12.Print Results:

Prints the results for both cross-validation and testing for each fold and threshold probability.

In [ ]:
print("\n\n\n\n")

thresh_probability = np.arange(start=0.3,stop=0.8,step=0.1)
fold_size = 123
K = testing_data_all_info.shape[0]//fold_size

for fold_idx in range(K):
    for p_thresh in thresh_probability:
        
        cv_data = pd.concat([testing_data_all_info[0:(fold_idx*fold_size)],
                             testing_data_all_info[(fold_idx+1)*fold_size:]],axis=0)
        
        testing_data = testing_data_all_info[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        
        multi_hot_encoded_Y_cv = np.concatenate((multi_hot_encoded_Y_test[0:(fold_idx*fold_size)],
                                                multi_hot_encoded_Y_test[(fold_idx+1)*fold_size:]),
                                                axis=0)
        
        multi_hot_encoded_Y_test = multi_hot_encoded_Y_test[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        X_cv, Y_cv, rescaled_centroid_cv = custom_testing_data_generator(cv_data,multi_hot_encoded_Y_cv,
                                                                         cv_data.shape[0])
        
        X_test, Y_test, rescaled_centroid_test = custom_testing_data_generator(testing_data,
                                                                               multi_hot_encoded_Y_test,
                                                                               testing_data.shape[0])
        
        Y_pred_cv = our_custom_cnn(X_cv)
        cv_acc,cv_pre,cv_rec = compute_performance_metrics(Y_pred_cv,Y_cv,p_thresh)
        
        cv_conv_base_out = conv_base(X_cv).numpy()
        cv_heatmaps = np.matmul(cv_conv_base_out,cls_head_weights)
        
        Y_pred_test = our_custom_cnn(X_test)
        test_acc,test_pre,test_rec = compute_performance_metrics(Y_pred_test,Y_test,p_thresh)
        
        test_conv_base_out = conv_base(X_test).numpy()
        test_heatmaps = np.matmul(test_conv_base_out,cls_head_weights)
        
        print("Threshold Probability: {}, CV Acc: {}, CV Prec: {}, CV Rec: {}".format(p_thresh,cv_acc,
                                                                                            cv_pre,
                                                                                            cv_rec))
        print("Threshold Probability: {}, Test Acc: {}, Test Prec: {}, Test Rec: {}".format(p_thresh,
                                                                                           test_acc,
                                                                                           test_pre,
                                                                                           test_rec))
        

In summary, this code performs cross-validation on a model using different threshold probabilities and prints the resulting performance metrics for both cross-validation and testing data. It also includes feature extraction and heatmap calculation steps. The output provides insights into the model's performance under different evaluation conditions.